In [37]:
import utils.common
import evaluation
import importlib
import numpy as np
import time
from models.ConstraintAutoRec import ConstraintAutoRec 
import models.NeuralLogicRec
import tensorflow as tf
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.NeuralLogicRec)
import itertools

ml_small = utils.common.ml_small

# ev = evaluation.Evaluation(ml_small)

In [38]:
nlr = models.NeuralLogicRec.NLR(ml_small['user'], ml_small['dimensions'], epochs=2, embedding_dim=8, batch_size=4)

In [39]:
nlr.train(utils.common.load_dataset(ml_small), ml_small['train']['records'])

Epoch #0 Loss at step 152: -401.2575, time: 87.0800
Epoch #1 Loss at step 152: -1948.8020, time: 75.284


In [40]:
# print(np.flip(np.argsort(nlr.predict_single_user(3))))
pred = nlr.model([1])
inf = models.NeuralLogicRec.map_inference(nlr.model, pred)
print(pred)
print(inf)
# nlr.model(np.array(1), np.array(1))
ev.evaluate(nlr)
# pred = nlr.model(tf.convert_to_tensor([1, 2, 3], dtype=tf.int32))

{'likes': <tf.Tensor: id=4437581, shape=(1, 10381), dtype=float32, numpy=
array([[0.100795  , 0.0747571 , 0.11193717, ..., 0.05833722, 0.12381735,
        0.13071798]], dtype=float32)>, 'rec': <tf.Tensor: id=4437627, shape=(1, 10381), dtype=float32, numpy=
array([[0.79961   , 0.8490398 , 0.74206924, ..., 0.8851336 , 0.7397918 ,
        0.75234365]], dtype=float32)>, 'sim': <tf.Tensor: id=4437656, shape=(1, 1), dtype=float32, numpy=array([[1.]], dtype=float32)>}
{'likes': <tf.Variable 'Variable:0' shape=(1, 10381) dtype=float32, numpy=array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, 'rec': <tf.Variable 'Variable:0' shape=(1, 10381) dtype=float32, numpy=array([[1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>, 'sim': <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[0.32990494]], dtype=float32)>}
Batch nr 1 predicted
waiting for queue
processing results
Evaluating NeuralLogicRec |============================================================>| 100.0% 


{'accuracy': 0.8484952120383037,
 'precision': 0.8484952120383037,
 'recall': 1.0,
 'map@1': 0.0,
 'map@5': 0.0008963585434173669,
 'map@10': 0.0012628384687208216,
 'diversity@5': 0.35761879449607004,
 'diversity@10': 0.36279122452761836,
 'epc@5': 0.9745420512673958,
 'epc@10': 0.9724472224779395,
 'epd@5': 0.3831683368104513,
 'name': 'NeuralLogicRec',
 'latent_dim': 128,
 'epochs': 2,
 'batch_size': 4}

In [ ]:
u = tf.convert_to_tensor([x for x in itertools.permutations(range(600), 2)])
u1 = u[:,0]
u2 = u[:,1]

In [ ]:
embed_u1 = tf.nn.embedding_lookup(nlr.model.user_embedding, u1)
embed_u2 = tf.nn.embedding_lookup(nlr.model.user_embedding, u2)

In [ ]:
def cosine_sim(a, b):
    cos_similarity = tf.keras.losses.cosine_similarity(a, b,axis=1)
    return ( 1 + cos_similarity) / 2
    
len(cosine_sim(embed_u1, embed_u2))



In [86]:
num_movies = 10
num_user = 3
rated = tf.convert_to_tensor(np.arange(num_movies * num_user).reshape([num_user, num_movies]) % 3 == 0)
seen = tf.convert_to_tensor(np.arange(num_movies * num_user, num_movies * num_user * 2).reshape([num_user, num_movies]))
users = tf.convert_to_tensor(np.arange(num_user))
users = tf.convert_to_tensor([20, 10, 5, 0, 40])
print(rated)
print(seen)

tf.Tensor(
[[ True False False  True False False  True False False  True]
 [False False  True False False  True False False  True False]
 [False  True False False  True False False  True False False]], shape=(3, 10), dtype=bool)
tf.Tensor(
[[30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]], shape=(3, 10), dtype=int64)


In [87]:
user_cross = tf.convert_to_tensor([x for x in itertools.permutations(users.numpy(), 2)])
print(user_cross)
rated_cross = tf.convert_to_tensor([tf.stack([rated[x[0]], rated[x[1]]], axis=0) for x in itertools.permutations(range(len(users.numpy())), 2)])
# print(rated_cross)

tf.Tensor(
[[20 10]
 [20  5]
 [20  0]
 [20 40]
 [10 20]
 [10  5]
 [10  0]
 [10 40]
 [ 5 20]
 [ 5 10]
 [ 5  0]
 [ 5 40]
 [ 0 20]
 [ 0 10]
 [ 0  5]
 [ 0 40]
 [40 20]
 [40 10]
 [40  5]
 [40  0]], shape=(20, 2), dtype=int32)


InvalidArgumentError: slice index 3 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/

In [88]:
len(user_cross)

20

In [89]:
user_emb = tf.expand_dims(tf.nn.embedding_lookup(nlr.model.user_embedding, user_cross), axis=2)
print(user_emb[:, 0])
user_embedd = tf.tile(user_emb, [1, 1, num_movies, 1])
expanded_embedd = tf.reshape(nlr.model.item_embedding, [1, 1, num_movies, 24])
item_embedd = tf.tile(expanded_embedd, [len(user_cross), 2, 1, 1])
print(item_embed.shape)
print(user_embedd.shape)
inp = tf.concat([user_embedd, item_embedd], axis=-1)

tf.Tensor(
[[[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.6136244   1.5436143   2.036422    1.5196469   0.08901728
   -0.90769035 -1.6418009  -0.3131709 ]]

 [[ 0.6136244   1.5436143   2.036422    1.5196469   

InvalidArgumentError: Input to reshape is a tensor with 83048 values, but the requested shape has 240 [Op:Reshape]

In [90]:
result = tf.squeeze(nlr.model.likes_estimator(inp))

NameError: name 'inp' is not defined

In [91]:
tf.reduce_mean(tf.square(tf.cast(rated_cross, tf.float32) - result))

NameError: name 'rated_cross' is not defined

In [92]:
tf.slice(user_cross, [0,0], [4, 1])

<tf.Tensor: id=2649591, shape=(4, 1), dtype=int32, numpy=
array([[20],
       [20],
       [20],
       [20]], dtype=int32)>

In [93]:
del nlr

In [94]:
print(tf.version)

<module 'tensorflow._api.v2.version' from '/home/markush/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [95]:
rated

<tf.Tensor: id=2649542, shape=(3, 10), dtype=bool, numpy=
array([[ True, False, False,  True, False, False,  True, False, False,
         True],
       [False, False,  True, False, False,  True, False, False,  True,
        False],
       [False,  True, False, False,  True, False, False,  True, False,
        False]])>

In [84]:
num_movies = 5
users = tf.convert_to_tensor([1, 10, 100])
embed_user = tf.expand_dims(users, axis=1)
users_1 = tf.tile(tf.expand_dims(embed_user, axis=1), [1, len(users), 1])
users_2 = tf.tile(tf.expand_dims(embed_user, axis=0), [len(users),1, 1])
# print(embed_user)
# embed_user = tf.tile(embed_user, [1, num_movies, 1])
# expanded_embedd = tf.expand_dims(nlr.model.item_embedding[0:num_movies], axis=0)
# embed_item = tf.tile(expanded_embedd, [len(users), 1, 1])
sim_input = tf.reduce_sum(tf.concat([users_1, users_2], axis=-1), axis=-1)
print(sim_input)

tf.Tensor(
[[  2  11 101]
 [ 11  20 110]
 [101 110 200]], shape=(3, 3), dtype=int32)


In [99]:
multi_ratings = tf.tile(tf.expand_dims(rated, axis=0), [3, 1, 1])

In [17]:
a = tf.expand_dims(tf.convert_to_tensor(10), axis=0)
b = tf.expand_dims(tf.convert_to_tensor(20), axis=0)
c = tf.convert_to_tensor([1,2,3,4])

In [18]:
tf.concat([a,b, c], axis=0)

<tf.Tensor: id=134207, shape=(6,), dtype=int32, numpy=array([10, 20,  1,  2,  3,  4], dtype=int32)>

In [20]:
tf.squeeze(tf.reshape(a, [1, 1, 1]), -1)

<tf.Tensor: id=134214, shape=(1, 1), dtype=int32, numpy=array([[10]], dtype=int32)>